# Part 9

Universal code that we will use for the entire notebook.

In [1]:
# Import necessary libraries 
from openai import OpenAI  # Used for interacting with OpenAI's API
from typing_extensions import override  # Used for overriding methods in subclasses
from openai import AssistantEventHandler # Used for handling events related to OpenAI assistants

# Additional libraries for time and date manipulation
import time
import pytz
import datetime 

In [2]:
# Create an instance of the OpenAI class to interact with the API.
# This assumes you have set the OPENAI_API_KEY environment variable.
client = OpenAI() 

## Creating Assistants, Threads, and Messages Review
Let's create a new assistant, thread, and some messages for us to use later on and to review the code for creating them.

### Creating an Assistant
First, let's make an Assistant we can use to communicate with our run.

In [3]:

# Create an assistant using the client library.
assistant = client.beta.assistants.create(
    model="gpt-4o",  # Specify the model to be used.
    instructions="You are a helpful assistant.",  # Set the instructions for the assistant.
    name="Son of Run Tester Assistant",  # Give the assistant a name.
    metadata={  # Add metadata about the assistant's capabilities.
        "holds_threads": "True",
        "likes_threads": "True",
        "holds_messages": "True",
        "likes_messages": "True",
    },
    temperature=1,  # Set the temperature for response variability.
    top_p=1,  # Set the top_p for nucleus sampling.
)

# Print the details of the created assistant to check its properties.
print(assistant)  # Print the full assistant object.
print("\n\n")
print(assistant.name)  # Print the name of the assistant.
print(assistant.metadata)  # Print the metadata of the assistant.


Assistant(id='asst_GQAyOA9rDwc6hLpxzIbTH9fm', created_at=1715845576, description=None, instructions='You are a helpful assistant.', metadata={'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True'}, model='gpt-4o', name='Son of Run Tester Assistant', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)



Son of Run Tester Assistant
{'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True'}


### Creating a Thread
Now, let's create a Thread that can be used to hold our messages.

In [4]:
# Create a thread using the OpenAI API and store it in a variable.
# The metadata specifies a user identifier.
thread = client.beta.threads.create(
    metadata={
        "user": "abc123"
    }
)

# Output the result of the thread creation to the console.
print(thread)


Thread(id='thread_DuiqEcj75jTFpNALdGumxQnb', created_at=1715845598, metadata={'user': 'abc123'}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


### Creating a Message
Finally, let's create a Message that we can go into the Thread for use later.

In [5]:
# Create a message in a specific thread using the client's message creation method.
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # ID of the thread where the message will be posted
    role="user",  # Role of the entity posting the message
    content="Tell me what a penguin is in 100 words or less.",  # The textual content of the message
    metadata={"key": "value"}  # Additional data associated with the message in key-value pairs
)

# Print the entire message object to view its details.
print(message)

# Print a blank line for better readability of the output.
print("\n")

# Print specific attributes of the message.
print(message.id)  # The unique identifier of the message
print(message.content)  # The content of the message
print(message.content[0].text.value)  # Assuming 'content' is a list of text objects, print the value of the first one
print(message.role)  # The role associated with the message

Message(id='msg_sSyqoMb4eYREEN4cEEFdoLom', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Tell me what a penguin is in 100 words or less.'), type='text')], created_at=1715845607, incomplete_at=None, incomplete_details=None, metadata={'key': 'value'}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_DuiqEcj75jTFpNALdGumxQnb')


msg_sSyqoMb4eYREEN4cEEFdoLom
[TextContentBlock(text=Text(annotations=[], value='Tell me what a penguin is in 100 words or less.'), type='text')]
Tell me what a penguin is in 100 words or less.
user


## Creating Streaming Run
Let's create a run and get some output!

In [6]:
# Create an EventHandler class to define how we want to handle the events in the response stream.
# Normally, you would define this class at the top of your script or in a separate file.

class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    @override
    def on_text_created(self, text) -> None:
        """Handle the event when text is first created.
        
        This method is called when a complete piece of text is generated by the assistant.
        """
        # Print a prompt indicating the start of assistant's text response.
        # 'flush=True' ensures that the output is immediately written to the console.
        print("\nassistant text > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text).
        
        This method is called for incremental text updates, useful for streaming responses.
        """
        # Print the incremental text as it is being generated.
        # 'end=""' prevents adding a new line after each delta.
        # 'flush=True' ensures that the output is immediately written to the console.
        print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
        """Handle the event when a tool call is created.
        
        This method is called when the assistant makes a call to an external tool.
        """
        # Print a prompt indicating the assistant is using a tool.
        # 'flush=True' ensures that the output is immediately written to the console.
        print(f"\nassistant tool > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        """Handle the event when there is a delta (update) in a tool call.
        
        This method is called for incremental updates during the tool call's execution.
        """
        if delta.type == 'code_interpreter':
            # If there is an input for the code interpreter, print it.
            # 'flush=True' ensures that the output is immediately written to the console.
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            # If there are outputs from the code interpreter, print them.
            if delta.code_interpreter.outputs:
                # 'flush=True' ensures that the output is immediately written to the console.
                print("\n\noutput >", flush=True)
            # Loop through each output and print the logs if present.
            for output in delta.code_interpreter.outputs:
                if output.type == "logs":
                    # 'flush=True' ensures that the output is immediately written to the console.
                    print(f"\n{output.logs}", flush=True)


In [7]:
# Use the `stream` SDK helper with the `EventHandler` class to create the Run
# and stream the response.
with client.beta.threads.runs.stream(
    thread_id=thread.id,         # ID of the thread to run.
    assistant_id=assistant.id,   # ID of the assistant to use.
    event_handler=EventHandler(),  # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()



assistant text > A penguin is a flightless seabird belonging to the Spheniscidae family, primarily found in the Southern Hemisphere, especially Antarctica. Known for their distinctive black and white plumage, penguins are adept swimmers, using their flipper-like wings to navigate underwater. They feed mainly on fish, squid, and krill. Penguins are highly social animals, often living in large colonies. They have various species, including the Emperor, King, and Adélie penguins. Adaptations like thick blubber and dense feathers help them survive harsh, cold environments. Penguins also exhibit unique behaviors, such as huddling for warmth and prolonged parental care.

## Token Management
We have two options for contolling the amount of tokens used:

max_prompt_tokens (integer or null)
Optional
The maximum number of prompt tokens that may be used over the course of the run. The run will make a best effort to use only the number of prompt tokens specified, across multiple turns of the run. If the run exceeds the number of prompt tokens specified, the run will end with status incomplete. 

max_completion_tokens (integer or null)
Optional
The maximum number of completion tokens that may be used over the course of the run. The run will make a best effort to use only the number of completion tokens specified, across multiple turns of the run. If the run exceeds the number of completion tokens specified, the run will end with status incomplete. 

Let's see it in action!

First we start with max_prompt_tokens to limit the input tokens that can be used.

In [8]:
# Let's start by seeing how many messages we have.
messages = client.beta.threads.messages.list(
    thread_id=thread.id  # ID of the thread to list messages from.
)

# Initialize a counter for the messages.
message_count = 0

# Loop through the messages, print the id and content, and count the messages.
for message in messages:
    print(message.id)      # Print the ID of the message.
    print(message.content)  # Print the content of the message.
    print("\n")             # Print a newline for readability.
    message_count += 1      # Increment the message count.

# Print the total number of messages.
print(f"Total number of messages: {message_count}")


msg_s9MACZG7h6NsLyOdf6g6B3MT
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird belonging to the Spheniscidae family, primarily found in the Southern Hemisphere, especially Antarctica. Known for their distinctive black and white plumage, penguins are adept swimmers, using their flipper-like wings to navigate underwater. They feed mainly on fish, squid, and krill. Penguins are highly social animals, often living in large colonies. They have various species, including the Emperor, King, and Adélie penguins. Adaptations like thick blubber and dense feathers help them survive harsh, cold environments. Penguins also exhibit unique behaviors, such as huddling for warmth and prolonged parental care.'), type='text')]


msg_sSyqoMb4eYREEN4cEEFdoLom
[TextContentBlock(text=Text(annotations=[], value='Tell me what a penguin is in 100 words or less.'), type='text')]


Total number of messages: 2


In [9]:
# First, let's list our runs to see what we have.
runs = client.beta.threads.runs.list(
    thread_id=thread.id  # ID of the thread to list runs from.
)

# Dump all the runs.
print(runs)
print("\n\n")

# Loop through the runs and print the id and status.
for run in runs:
    print(run.id)      # Print the ID of the run.
    print(run.status)  # Print the status of the run.
    print("\n")        # Print a newline for readability.


SyncCursorPage[Run](data=[Run(id='run_ydOBWX453Qpf3ztSzho642fm', assistant_id='asst_GQAyOA9rDwc6hLpxzIbTH9fm', cancelled_at=None, completed_at=1715845805, created_at=1715845802, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=1715845802, status='completed', thread_id='thread_DuiqEcj75jTFpNALdGumxQnb', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=133, prompt_tokens=44, total_tokens=177), temperature=1.0, top_p=1.0, tool_resources={})], object='list', first_id='run_ydOBWX453Qpf3ztSzho642fm', last_id='run_ydOBWX453Qpf3ztSzho642fm', has_more=False)



run_ydOBWX453Qpf3ztSzho642fm
completed




In [15]:
# Now let's do a run limiting the prompt tokens.
with client.beta.threads.runs.stream(
    thread_id=thread.id,          # ID of the thread to run.
    assistant_id=assistant.id,    # ID of the assistant to use.
    # max_prompt_tokens=5,        # Limit the maximum number of prompt tokens to get an error
    max_prompt_tokens=256,        # Limit the maximum number of prompt tokens.
    event_handler=EventHandler(), # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()


In [12]:
# Let's list our runs again to see what we have.
runs = client.beta.threads.runs.list(
    thread_id=thread.id  # ID of the thread to list runs from.
)

# Dump all the runs.
print(runs)
print("\n\n")

# Loop through the runs and print the id and status.
for run in runs:
    print(run.id)      # Print the ID of the run.
    print(run.status)  # Print the status of the run.
    print("\n")        # Print a newline for readability.


SyncCursorPage[Run](data=[Run(id='run_JoaOm2eQGZMXzuqSS8Ghjsde', assistant_id='asst_GQAyOA9rDwc6hLpxzIbTH9fm', cancelled_at=None, completed_at=1715845963, created_at=1715845960, expires_at=None, failed_at=None, incomplete_details=IncompleteDetails(reason='max_prompt_tokens'), instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=256, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=1715845960, status='incomplete', thread_id='thread_DuiqEcj75jTFpNALdGumxQnb', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=122, prompt_tokens=179, total_tokens=301), temperature=1.0, top_p=1.0, tool_resources={}), Run(id='run_ydOBWX453Qpf3ztSzho642fm', assistant_id='asst_GQAyOA9rDwc6hLpxzIbTH9fm', cancelled_at=None, completed_at=1715845805, created_at=1715845802, expires_at=None, failed_at=None, incomplete_detai

And now limiting the completion tokens

In [13]:
# Now let's do a run limiting the completion tokens.
with client.beta.threads.runs.stream(
    thread_id=thread.id,         # ID of the thread to run.
    assistant_id=assistant.id,   # ID of the assistant to use.
    # max_completion_tokens=16,    # Limit the maximum number of completion tokens to get an error
    max_completion_tokens=16,    # Limit the maximum number of completion tokens.
    event_handler=EventHandler(),  # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()



assistant text > A penguin is a flightless seabird from the Spheniscidae

In [14]:
# Now let's list our runs again
runs = client.beta.threads.runs.list(
    thread_id=thread.id
)


# Loop throught the runs and print the id and status
for run in runs:
    print(run.id)
    print(run.status)
    print(run.incomplete_details)
    print("\n")

run_M7Y3X5dseTOuhgW5N8yhcavk
incomplete
IncompleteDetails(reason='max_completion_tokens')


run_JoaOm2eQGZMXzuqSS8Ghjsde
incomplete
IncompleteDetails(reason='max_prompt_tokens')


run_ydOBWX453Qpf3ztSzho642fm
completed
None




### Truncation Strategy
We will need to manage having a lot of context being sent via messages and managing the tokens sent each time.


truncation_strategy (object)
Optional
Controls for how a thread will be truncated prior to the run. Use this to control the intial context window of the run.

Properties:
type (string)
Required
The truncation strategy to use for the thread. The default is auto. If set to last_messages, the thread will be truncated to the n most recent messages in the thread. When set to auto, messages in the middle of the thread will be dropped to fit the context length of the model, max_prompt_tokens.

last_messages (integer or null)
Optional
The number of most recent messages from the thread when constructing the context for the run.

In [16]:
# Now let's use a truncation strategy on the last set of messages.
with client.beta.threads.runs.stream(
    thread_id=thread.id,           # ID of the thread to run.
    assistant_id=assistant.id,     # ID of the assistant to use.
    max_prompt_tokens=1000,        # Limit the maximum number of prompt tokens.
    truncation_strategy={          # Adding a truncation strategy to control the context of the conversation.
        "type": "last_messages",   # Strategy to keep only the most recent messages.
        "last_messages": 5         # Keep the last 5 messages in the context window.
    },
    event_handler=EventHandler(),  # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()



assistant text > A penguin is a flightless seabird from the Spheniscidae family, primarily found in the Southern Hemisphere, especially Antarctica. Recognizable by their black and white plumage, penguins are excellent swimmers, using their flipper-like wings underwater. They primarily consume fish, squid, and krill. Social in nature, penguins often gather in large colonies. There are various species, including the Emperor, King, and Adélie penguins. They possess adaptations like thick blubber and dense feathers to endure cold environments. Notable behaviors include huddling for warmth and extended parental care.

In [17]:
# Now let's use a truncation strategy where the system decides what messages to cut.
with client.beta.threads.runs.stream(
    thread_id=thread.id,           # ID of the thread to run.
    assistant_id=assistant.id,     # ID of the assistant to use.
    max_prompt_tokens=1000,        # Limit the maximum number of prompt tokens.
    truncation_strategy={          # Adding a truncation strategy to control the context of the conversation.
        "type": "auto"             # Strategy where the system automatically decides which messages to keep.
    },
    event_handler=EventHandler(),  # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()



assistant text > A penguin is a flightless seabird from the Spheniscidae family, primarily found in the Southern Hemisphere, especially Antarctica. Recognizable by their black and white plumage, penguins are excellent swimmers, using their flipper-like wings underwater. They primarily consume fish, squid, and krill. Social in nature, penguins often gather in large colonies. There are various species, including the Emperor, King, and Adélie penguins. They possess adaptations like thick blubber and dense feathers to endure cold environments. Unique behaviors include huddling for warmth and extended parental care.

In [18]:
# Let's see how many messages we have now
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# Initialize a counter for the messages
message_count = 0

# Loop through the messages, print the id and content, and count the messages
for message in messages:
    print(message.id)
    print(message.content)
    print("\n")
    message_count += 1  # Increment the message count

# Print the total number of messages
print(f"Total number of messages: {message_count}")


msg_MhiAECGm10oHLcXv6SssXX2a
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird from the Spheniscidae family, primarily found in the Southern Hemisphere, especially Antarctica. Recognizable by their black and white plumage, penguins are excellent swimmers, using their flipper-like wings underwater. They primarily consume fish, squid, and krill. Social in nature, penguins often gather in large colonies. There are various species, including the Emperor, King, and Adélie penguins. They possess adaptations like thick blubber and dense feathers to endure cold environments. Unique behaviors include huddling for warmth and extended parental care.'), type='text')]


msg_1x42n9UjyvEtayM9kPXckoe7
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird from the Spheniscidae family, primarily found in the Southern Hemisphere, especially Antarctica. Recognizable by their black and white plumage, penguins are excellent swimmers, using th

## Controlling Tool Choice and Response Format
In addition to all the other arguments we can modify, we also have tool_choice and response_format:

tool_choice (string or object)

Optional

Controls which (if any) tool is called by the model. none means the model will not call any tools and instead generates a message. auto is the default value and means the model can pick between generating a message or calling one or more tools. required means the model must call one or more tools before responding to the user. Specifying a particular tool like {"type": "file_search"} or {"type": "function", "function": {"name": "my_function"}} forces the model to call that tool.

response_format (string or object)

Optional

Specifies the format that the model must output. Compatible with GPT-4o, GPT-4 Turbo, and all GPT-3.5 Turbo models since gpt-3.5-turbo-1106.

Setting to { "type": "json_object" } enables JSON mode, which guarantees the message the model generates is valid JSON.

Important: when using JSON mode, you must also instruct the model to produce JSON yourself via a system or user message. Without this, the model may generate an unending stream of whitespace until the generation reaches the token limit, resulting in a long-running and seemingly "stuck" request. Also note that the message content may be partially cut off if finish_reason="length", which indicates the generation exceeded max_tokens or the conversation exceeded the max context length.



In [19]:
# Use the `stream` SDK helper with the `EventHandler` class to create the Run
# and stream the response.
with client.beta.threads.runs.stream(
    thread_id=thread.id,  # ID of the thread to run.
    assistant_id=assistant.id,  # ID of the assistant to use.
    additional_instructions="Make sure your output is in JSON format.",  # Instructions for JSON output.
    response_format={"type": "json_object"},  # Specify response format as JSON object.
    tool_choice=None,
    event_handler=EventHandler(),  # Custom event handler for processing events.
) as stream:
    stream.until_done()  # Process the stream until complete.



assistant text > {
  "description": "A penguin is a flightless seabird from the Spheniscidae family, primarily found in the Southern Hemisphere, especially Antarctica. Recognizable by their black and white plumage, penguins are excellent swimmers, using their flipper-like wings underwater. They primarily consume fish, squid, and krill. Social in nature, penguins often gather in large colonies. There are various species, including the Emperor, King, and Adélie penguins. They possess adaptations like thick blubber and dense feathers to endure cold environments. Unique behaviors include huddling for warmth and extended parental care."
}

## Listing Runs
thread_id
(string)

Required
The ID of the thread the run belongs to.

limit
(integer)

Optional
Defaults to 20
A limit on the number of objects to be returned. Limit can range between 1 and 100, and the default is 20.

order
(string)

Optional
Defaults to desc
Sort order by the created_at timestamp of the objects. asc for ascending order and desc for descending order.

after
(string)

Optional
A cursor for use in pagination. after is an object ID that defines your place in the list. For instance, if you make a list request and receive 100 objects, ending with obj_foo, your subsequent call can include after=obj_foo in order to fetch the next page of the list.

before
(string)

Optional
A cursor for use in pagination. before is an object ID that defines your place in the list. For instance, if you make a list request and receive 100 objects, ending with obj_foo, your subsequent call can include before=obj_foo in order to fetch the previous page of the list.

In [20]:
# List our runs to see what we have
runs = client.beta.threads.runs.list(
    thread_id=thread.id,
    order="desc",  # Order runs in descending order
)

# Loop through the runs and print the id, status, and incomplete details
for run in runs:
    print(run.id)
    print(run.status)
    print(run.incomplete_details)
    print("\n")



run_wfdnEzSVPZ90Q7JKAhpK1SPg
completed
None


run_2naxlt3UFgtdfVkf7jnHzWId
completed
None


run_dsKOymAtfCKdYU7r9c2mqnH9
completed
None


run_OfASG2aCcpcFspNMMp6muGL1
failed
None


run_M7Y3X5dseTOuhgW5N8yhcavk
incomplete
IncompleteDetails(reason='max_completion_tokens')


run_JoaOm2eQGZMXzuqSS8Ghjsde
incomplete
IncompleteDetails(reason='max_prompt_tokens')


run_ydOBWX453Qpf3ztSzho642fm
completed
None




## Retrieve Runs

thread_id
(string)

Required
The ID of the thread that was run.

run_id
(string)

Required
The ID of the run to retrieve.

In [21]:
# List our runs to see what we have
runs = client.beta.threads.runs.list(
    thread_id=thread.id,
    order="desc",  # Order runs in descending order
)

# Convert the normal SyncCursorPage object we get back to an actual list
runs_list = list(runs)

# Check if there are any runs
if runs_list:
    # Get the run id of the last run (first in the list since it's sorted in descending order)
    last_run_id = runs_list[0].id
    print("Last run ID:", last_run_id)

# Loop through the runs and print the id, status, and incomplete details
for run in runs_list:
    print(run.id)
    print(run.status)
    print(run.incomplete_details)
    print("\n")



Last run ID: run_wfdnEzSVPZ90Q7JKAhpK1SPg
run_wfdnEzSVPZ90Q7JKAhpK1SPg
completed
None


run_2naxlt3UFgtdfVkf7jnHzWId
completed
None


run_dsKOymAtfCKdYU7r9c2mqnH9
completed
None


run_OfASG2aCcpcFspNMMp6muGL1
failed
None


run_M7Y3X5dseTOuhgW5N8yhcavk
incomplete
IncompleteDetails(reason='max_completion_tokens')


run_JoaOm2eQGZMXzuqSS8Ghjsde
incomplete
IncompleteDetails(reason='max_prompt_tokens')


run_ydOBWX453Qpf3ztSzho642fm
completed
None




In [22]:
# Retrieve the details of a specific run
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=runs_list[0].id  # Use the ID of the first run in the list that was sorted descending and therefore is the last run to happen
)

# Print the details of the retrieved run
print(run)



Run(id='run_wfdnEzSVPZ90Q7JKAhpK1SPg', assistant_id='asst_GQAyOA9rDwc6hLpxzIbTH9fm', cancelled_at=None, completed_at=1715846478, created_at=1715846475, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant. Make sure your output is in JSON format.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format=AssistantResponseFormat(type='json_object'), started_at=1715846475, status='completed', thread_id='thread_DuiqEcj75jTFpNALdGumxQnb', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=128, prompt_tokens=584, total_tokens=712), temperature=1.0, top_p=1.0, tool_resources={})


## Create Thread and Run
We have a helper function that can be used to create a thread and run it in one shot.

assistant_id
(string)

Required

The ID of the assistant to use to execute this run.




In [25]:
# Create and run a thread with a specific message
run = client.beta.threads.create_and_run(
    assistant_id=assistant.id,  # ID of the assistant to use.
    thread={
        "messages": [
            {
                "role": "user", 
                "content": "Explain deep learning to me like I'm a 5 year old. In 50 words or less."
            }
        ]
    }
)

# Print the run details
print(run)
print("\n")

# Print the thread ID of the run
print(run.thread_id)
print("\n")

# Sleep for a few seconds to allow the run to complete
time.sleep(10)

# Retrieve the latest status of the run
run = client.beta.threads.runs.retrieve(
    thread_id=run.thread_id,  # Use the thread ID from the created run.
    run_id=run.id  # Use the run ID from the created run.
)

# Print the run ID and status
print(run.id)
print(run.status)
print("\n")

# Get the messages from the thread, ordered in ascending order
messages = client.beta.threads.messages.list(
    thread_id=run.thread_id,  # Use the thread ID from the run.
    order="asc"  # Order messages in ascending order
)

# Loop through the messages and print the content
for message in messages:
    print(message.content)
    print("\n")


Run(id='run_p9bOoTvOs5fEEOjBMBpD02oF', assistant_id='asst_GQAyOA9rDwc6hLpxzIbTH9fm', cancelled_at=None, completed_at=None, created_at=1715846869, expires_at=1715847469, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_oRArZDdlArpjwpOe0mjPZS5S', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


thread_oRArZDdlArpjwpOe0mjPZS5S


run_p9bOoTvOs5fEEOjBMBpD02oF
completed


[TextContentBlock(text=Text(annotations=[], value="Explain deep learning to me like I'm a 5 year old. In 50 words or less."), type='text')]


[TextContentBlock(text=Text(annotations=[], value='Deep learning is like teaching a computer to be smart by showing it l

## Modifying Runs
You can only modify the metadata on runs at this time.

thread_id
(string)

Required
The ID of the thread that was run.

run_id
(string)

Required
The ID of the run to modify.

Request body
(metadata)
map

Optional
Set of 16 key-value pairs that can be attached to an object. This can be useful for storing additional information about the object in a structured format. Keys can be a maximum of 64 characters long and values can be a maxium of 512 characters long.

In [26]:
# Show our run information before modification
print(run.id)  # Print the run ID
print(run.metadata)  # Print the metadata of the run
print("\n")

# Update the run's metadata
run = client.beta.threads.runs.update(
    thread_id=run.thread_id,  # Use the thread ID from the run
    run_id=run.id,  # Use the run ID
    metadata={"user_id": "user_abc123"},  # New metadata to update
)

# Show our run information after modification
print(run.id)  # Print the run ID
print(run.metadata)  # Print the updated metadata of the run



run_p9bOoTvOs5fEEOjBMBpD02oF
{}


run_p9bOoTvOs5fEEOjBMBpD02oF
{'user_id': 'user_abc123'}


## Canceling Runs
You can cancel any run that has a state of in_progress at any time. 

thread_id
(string)

Required
The ID of the thread to which this run belongs.

run_id
(string)

Required
The ID of the run to cancel.

In [27]:
import datetime
import pytz

# Create a new run with the stream option set to True
stream_run = client.beta.threads.runs.create(
    thread_id=thread.id,  # ID of the thread to run.
    assistant_id=assistant.id,  # ID of the assistant to use.
    stream=True  # Enable streaming for the run.
)

# Variable to hold the cancelled run object
cancelled_run = None

# Continue processing the new run
for event in stream_run:
    # Check if 'status' is an attribute of the event data
    if hasattr(event.data, 'status'):
        print(event.data.id)  # Print the event ID
        print(event.data.status)  # Print the event status

        # Cancel the run if it is in progress
        if event.data.status == "in_progress":
            print("Cancelling the run.")
            cancelled_run = client.beta.threads.runs.cancel(
                thread_id=thread.id,  # ID of the thread
                run_id=event.data.id  # ID of the run to cancel
            )
    else:
        print(f"Event ID: {event.data.id} does not have a status attribute.")
        print(event.data.delta)  # Print the delta of the event data if no status attribute
    print("---------------\n")

# Retrieve the latest status of the cancelled run
cancelled_run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,  # ID of the thread
    run_id=cancelled_run.id  # ID of the cancelled run
)

# Show the run ID and when it was cancelled
print("=============== Cancelled Run Information ================\n")
print(cancelled_run.id)  # Print the run ID
print(cancelled_run.status)  # Print the status of the cancelled run
print(cancelled_run.cancelled_at)  # Print the cancellation timestamp

# Convert the created date/time in Unix format directly to Central Time and format it
formatted_date = datetime.datetime.fromtimestamp(cancelled_run.created_at, tz=pytz.utc) \
    .astimezone(pytz.timezone('America/Chicago')) \
    .strftime('%Y-%m-%d %I:%M:%S %Z')

# Print the formatted date and time
print(formatted_date)



run_yeSCEVIyhFLYSBrAbBThbY7l
queued
---------------

run_yeSCEVIyhFLYSBrAbBThbY7l
queued
---------------

run_yeSCEVIyhFLYSBrAbBThbY7l
in_progress
Cancelling the run.
---------------

run_yeSCEVIyhFLYSBrAbBThbY7l
cancelling
---------------

run_yeSCEVIyhFLYSBrAbBThbY7l
cancelled
---------------

=============== Cancelled Run Information ================

run_yeSCEVIyhFLYSBrAbBThbY7l
cancelled
1715846952
2024-05-16 03:09:11 CDT
